In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from itertools import combinations
from collections import Counter, defaultdict
from astropy.table import Table, join, vstack, unique
from astropy.coordinates import SkyCoord, Distance
from SAGA.utils import add_skycoord, find_near_objid, find_near_ra_dec, fill_values_by_query
from easyquery import Query, QueryMaker

import SAGA
from SAGA.database import FitsTable, GoogleSheets
from SAGA import ObjectCuts as C
from SAGA.utils import add_skycoord
from SAGA.utils.distance import z2v, d2m, m2d
from SAGA.utils.display import *
print(SAGA.__version__)

In [ ]:
saga = SAGA.QuickStart()
saga.set_default_base_version("paper2")

In [ ]:
def annotate_base(base):
    base["Mr_"] = base["r_mag"] - d2m(base["HOST_DIST"])
    base["Mr"] = np.where(np.isfinite(base["Mr"]) & C.has_spec.mask(base), base["Mr"], base["Mr_"])
    base["log_sm"] = np.where(np.isfinite(base["log_sm"]), base["log_sm"], 1.254 + 1.0976 * base["gr"] - 0.4 * base["Mr"])
    
    base["human_selected"] = 0
    hs = saga.database["human_selected"].read()
    for i in range(1, 4):
        base["human_selected"] += np.in1d(base["OBJID"], Query(f"score == {i}").filter(hs, "OBJID")).astype(np.int) * i

    return base

In [ ]:
base = SAGA.database.FitsTable("/home/yymao/Downloads/saga_base_all.fits").read()
base = saga.host_catalog.construct_host_query("paper2_complete").filter(base)
assert len(np.unique(base["HOSTID"])) == 36

base = annotate_base(base)

In [ ]:
assert not Query(C.is_sat, ~C.paper2_targeting_cut).mask(base).any()

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(10, 4))

def add_points(*queries, **plot_options):
    t = Query(*queries).filter(base, ["r_mag", "sb_r", "gr"])
    plot_options.setdefault("rasterized", True)
    plot_options.setdefault("lw", 0)
    ax[1].scatter(t["r_mag"], t["gr"], **plot_options)
    if "label" in plot_options:
        del plot_options["label"]
    ax[0].scatter(t["r_mag"], t["sb_r"], **plot_options)    

is_p1_sats = Query(C.is_sat, saga.host_catalog.construct_host_query("paper1_complete"), QueryMaker.in1d("OBJID", [4443380000000000475, 3115540000000003218], invert=True))
    
add_points(~C.has_spec, c="grey", s=0.05, label="Galaxies, w/o $z$")
add_points(C.has_spec, ~C.is_low_z, c="C0", s=0.1, label="Galaxies, $z>0.02$")
add_points(C.has_spec, C.is_low_z, ~C.is_sat, c="C1", marker='o', s=5, alpha=1, label="Galaxies $z<0.02$")

add_points(is_p1_sats, c="C2", marker='^', s=14, label="Sat. (Paper I)", lw=1, zorder=99)
add_points(Query(C.is_sat, ~is_p1_sats), c="C2", marker='s', s=10, label="Sat. (This work)", lw=1, zorder=99)
add_points(Query("human_selected >= 2", C.has_been_targeted, ~C.has_spec),  edgecolors="C5", c="None", marker='o', s=10, alpha=0.8, lw=1, label="dLSBGs, w/o $z$")

r = np.linspace(11.8, 21, 91)
ax[0].plot(r, 0.7*(r-14) + 18.5, color='C4', ls="--", alpha=1, label='Primary targeting region')
ax[1].plot(r, -0.06*(r-14) + 0.9, color='C4', ls="--", alpha=1)

ax[0].set_xlim(11.8, 21)
ax[1].set_xlim(11.8, 21)

ax[0].set_ylim(18, 26.5)
ax[1].set_ylim(-0.05, 1.55)

ax[1].set_xlabel(r'$r_o$')
ax[1].set_ylabel(r'$(g-r)_o$')
ax[0].set_xlabel(r'$r_o$')
ax[0].set_ylabel(r'${\mu}_{r_o,{\rm eff}}$')

lgnd = ax[0].legend(loc="lower right", scatterpoints=3, frameon=True, fontsize=12.5, handletextpad=0.5)

lgnd = ax[1].legend(loc="upper left", scatterpoints=3, frameon=True, fontsize=12.5, ncol=2, handletextpad=0.5, columnspacing=1)
lgnd.legendHandles[0]._sizes = [10]
lgnd.legendHandles[1]._sizes = [10]
lgnd.legendHandles[2]._sizes = [15]
lgnd.legendHandles[3]._sizes = [15]

fig.tight_layout()
plt.savefig('/home/yymao/Downloads/targeting-cuts.pdf', dpi=200, bbox_inches='tight')